In [ ]:
import copy

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
from landlab.io import read_esri_ascii
from landlab.components import PriorityFloodFlowRouter
from landlab.components import ErosionDeposition, FlowAccumulator
from landlab.plot import imshow_grid

import matplotlib.pyplot as plt
import numpy as np
from matplotlib import cm

from landlab import HexModelGrid, RasterModelGrid
from landlab.components import (
    DepressionFinderAndRouter,
    FlowAccumulator,
    FlowDirectorD8,
    FlowDirectorSteepest,
)
from landlab.plot.drainage_plot import drainage_plot



# --- Read elevation grid from ESRI ASCII ---
rmg, z = read_esri_ascii("hugo_site.asc", name="topographic__elevation")
rmg.status_at_node[z < 0.0] = rmg.BC_NODE_IS_CLOSED


# this is the same as writing:
fa = PriorityFloodFlowRouter(
    rmg,
    surface="topographic__elevation",
    flow_metric="D8",
    update_flow_depressions=True,
    runoff_rate=None,
    depression_handler="breach",
)
fa.run_one_step()

plt.figure()
drainage_plot(rmg)

K = 0.0001  # erodibility coefficient, 1/yr
m_sp = 0.5  # exponent on drainage area or discharge, -
n_sp = 1.0  # exponent on slope, -
sp_crit = 0.0  # erosion threshold
v_s = 100.0  # settling velocity parameter (dimensionless if drainage area is used instead of discharge)
F_f = 0.5  # fraction of fines generated during bed erosion
initial_elevation = (
    200.0  # starting elevation of an "uplifted block" (rapid baselevel drop), m
)

run_duration = 1000  # duration of run, yr
dt = 10  # time-step duration, yr
plot_every = 100  # time interval for plotting, yr

# Derived parameters
nsteps = int(run_duration / dt)
next_plot = plot_every

# set up colormap
cmap = copy.copy(mpl.colormaps["pink"])

ed = ErosionDeposition(
    rmg,
    K=K,
    m_sp=m_sp,
    n_sp=n_sp,
    sp_crit=sp_crit,
    v_s=v_s,
    F_f=F_f,
    solver="adaptive",  # use the adaptive time stepper, which is slightly faster
)

for i in range(1, nsteps + 1):
    # route flow
    fa.run_one_step()  # run_one_step isn't time sensitive, so it doesn't take dt as input

    # do some erosion/deposition
    ed.run_one_step(dt)

    if i * dt >= next_plot:
        plt.figure()
        imshow_grid(
            rmg,
            "topographic__elevation",
            grid_units=["m", "m"],
            var_name="Elevation (m)",
            cmap=cmap,
        )
        next_plot += plot_every
